In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial='https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/archive'
url_root= 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/archive'            
r = requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/686',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/658',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/634',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/608',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/577',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/554',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/17',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/16',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/15',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/14',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/13',
 'https://revistas.juridicas.unam.mx/index.php/derecho-inte

In [5]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='issues media-list')
    volumen=box.findAll('div', class_='issue-summary media')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [6]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('strong > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Estoy en la pagina https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/archive


In [7]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

23

In [8]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [9]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Anuario Mexicano de Derecho Internacional'
    a='V. Ciencias Sociales'
    tem='Derecho Internacional'
    url='http://www.revistas.unam.mx/index.php/amdi/about/editorialTeam'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('li', class_='active').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [10]:
list_scraper=list_scraper[0:23]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22


In [11]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,"Volumen XXII, enero-diciembre 2022",None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
1,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,"Volumen XXI, enero-diciembre 2021",None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
2,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,"Volumen XX, enero-diciembre 2020",None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
3,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,"Volumen XIX, enero-diciembre 2019",None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
4,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,"Volumen XVIII, enero-diciembre 2018",None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
5,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,"Volumen XVII, enero-diciembre 2017",None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
6,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,"Volumen XVI, enero-diciembre 2016",None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
7,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Volumen XV,None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
8,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Volumen XIV,None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...
9,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Volumen XIII,None,http://www.revistas.unam.mx/index.php/amdi/abo...,https://revistas.juridicas.unam.mx/index.php/d...


In [13]:
df_catalogo.to_csv('Revista25.csv', index=False)